In [11]:
import requests
import re
import json
import time
import datetime
from math import *
import sys
import functools
import warnings
warnings.filterwarnings("ignore")

In [12]:
# Retrieve login parameters from config file
with open("config.json") as credentials:
    credentials = json.load(credentials)

client_id = credentials["client_id"]
client_secret = credentials["client_secret"]

In [13]:
class RequestError(Exception):
    
    def __init__(self, response, message):
        self.response = response
        self.message = message
        super().__init__(message)

class StatusError(Exception):
    
    def __init__(self, status, message):
        self.response = response
        self.message = message
        super().__init__(message)

In [35]:
class JourneyBuilder:

    def __init__(self, client_id, client_secret, journey_name):
        # the function that is executed when
        # an instance of the class is created
        self.client_id = client_id
        self.client_secret = client_secret
        self.journey_name = journey_name
        
        self.auth_url = "https://mc42bdlx7mz5h4np2xxvhsb4scvq.auth.marketingcloudapis.com/v2/token"
        self.auth_headers = {"content-type": "application/json"}
        self.auth_payload = {
            "grant_type": "client_credentials",
            "client_id": self.client_id,
            "client_secret": self.client_secret
        }
        
        self.rest_url = "https://mc42bdlx7mz5h4np2xxvhsb4scvq.rest.marketingcloudapis.com"

        try:
            self.access_token, self.access_token_expiration, self.rest_headers = self.getAccessToken()
            if self.access_token is None:
                raise Exception("Request for access token failed.")
        except Exception as e:
            print(e)

    def getAccessToken(self):
        # the function that is 
        # used to request the JWT
        try:
            # request an access token
            authentication_response = requests.post(
                url=self.auth_url, data=json.dumps(self.auth_payload), headers=self.auth_headers, verify=False)

            # optional: raise exception for status code
            authentication_response.raise_for_status()
        except Exception as e:
            print(e)
            return None
        else:
            # assuming the response's structure is
            # {"access_token": ""}
            access_token = authentication_response.json()["access_token"]
            access_token_expiration = authentication_response.json()["expires_in"]
            rest_headers = {"authorization": f"Bearer {access_token}"}
            return access_token, access_token_expiration, rest_headers
    
    class Decorators():
        @staticmethod
        def refreshToken(decorated):
            # Decorator function
            # If func (e.g. pause_journey_version) is passed to @Decorators.refreshToken
            # Then token gets refreshed if expired
            # Wrapper returns func response
            def wrapper(api,*args,**kwargs):
                if time.time() > api.access_token_expiration:
                    api.access_token, api.access_token_expiration, api.rest_headers = api.getAccessToken()
                return decorated(api,*args,**kwargs)

            return wrapper

    @Decorators.refreshToken
    def get_metadata(self, *args):
        # Request gets all the journeys in a dict
        # Get values when name key == "journey_name"
        r = requests.get(
            url=f"{self.rest_url}/interaction/v1/interactions/",
            headers=self.rest_headers,
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(r.status_code))
        if not match:
            raise RequestError(
                r, "get_metadata: Response not in 200s.")

        return [[journey[arg] for journey in json.loads(r.content)["items"]
                 if journey["name"] == self.journey_name][0] for arg in args]
    
    @Decorators.refreshToken
    def get_activities_by_id(self, journey_id):
        # Request gets all activities within journey based on id
        r = requests.get(
            url=f"{self.rest_url}/interaction/v1/interactions/{journey_id}",
            headers=self.rest_headers,
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(r.status_code))
        if not match:
            raise RequestError(
                r, "get_activities_by_id: Response not in 200s.")
        
        return json.loads(r.content)
    
    @Decorators.refreshToken
    def change_status(self, journey_id, journey_version, action):
        # API needs journey id + version to pause
        r = requests.post(
            url=f"{self.rest_url}/interaction/v1/interactions/{action}/{journey_id}?versionNumber={journey_version}",
            headers=self.rest_headers,
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(r.status_code))
        if not match:
            raise RequestError(
                r, f"change_status: Response not in 200s.")
    
    @Decorators.refreshToken
    def update_journey_version(self, journey_id, journey_version, new_perc):
        # Pause journey version
        self.change_status(journey_id, journey_version, "pause")
        print(f"Journey: {journey_id, journey_version} PAUSED SUCCESSFULLY!")
        
        # Update journey version
        journey_activities = self.get_activities_by_id(journey_id)
        new_version_journey_activities = self.update_journey_perc(journey_activities, new_perc)
        r = requests.put(
            url=f"{self.rest_url}/interaction/v1/interactions",
            headers=self.rest_headers,
            data=json.dumps(new_version_journey_activities),
            verify=False
        )
        # Raise exception if status not in 200s
        pattern = re.compile("20[0-9]")
        match = re.findall(pattern, str(r.status_code))
        if not match:
            raise RequestError(
                r, f"update_journey_version: Response not in 200s.")
        print(f"Journey: {journey_id, journey_version} UPDATED SUCCESSFULLY!")
            
        # status needs to be "Paused" and not "Pausing" 
        # API requires some time to do that
        # Solution: Raise error is status != "Paused" or while loop until status == "Paused"
        time.sleep(300)
        journey_activities = self.get_activities_by_id(journey_id)
        status = journey_activities["status"]
        if status != "Paused":
            raise StatusError(
                status, f"status = {status}, should be Paused.")        
        
        # Resume journey version
        self.change_status(journey_id, journey_version, "resume")
        print(f"Journey: {journey_id, journey_version} RESUMED SUCCESSFULLY!")
    
    @Decorators.refreshToken
    def create_new_journey_version(self, journey_id, journey_version, new_perc):
        # Update journey version
        journey_activities = self.get_activities_by_id(journey_id)
        new_version_journey_activities = self.update_journey_perc(journey_activities, new_perc, version="new")
        # Create new jounrey version
        r = requests.post(
            url=f'{self.rest_url}/interaction/v1/interactions',
            headers=self.rest_headers,
            data=json.dumps(new_version_journey_activities),
            verify=False
        )
        print(f"Journey: {journey_id, journey_version} CREATED SUCCESSFULLY!")
        # Stop old version
        self.change_status(journey_id, journey_version, "stop")
        print(f"Journey: {journey_id, journey_version} STOPPED SUCCESSFULLY!")
            
    @staticmethod
    def update_journey_perc(journey_activities, new_perc, version='old'):
        # "definitionId" cannot be in data for it to work
        if version == 'new':
            del journey_activities["definitionId"]
        # Update journey version data with new perc
        journey_activities["activities"][0]["outcomes"][0]["arguments"]["percentage"] = str(new_perc)
        journey_activities["activities"][0]["outcomes"][0]["metaData"]["label"] = str(new_perc) + "%"
        journey_activities["activities"][0]["outcomes"][1]["arguments"]["percentage"] = str(100-new_perc)
        journey_activities["activities"][0]["outcomes"][1]["metaData"]["label"] = str(100-new_perc) + "%"
        return journey_activities

In [36]:
test = JourneyBuilder(client_id, client_secret, "test_2910") # "test_kwr_multilanguage_setup")

In [37]:
id, version, status = test.get_metadata("id", "version", "status")
id, version, status

('e1906618-b1e6-4649-a159-1e382cc483c6', 1, 'Published')

In [27]:
test.update_journey_version("e1906618-b1e6-4649-a159-1e382cc483c6", 1, 39)

Journey: ('e1906618-b1e6-4649-a159-1e382cc483c6', 1) PAUSED SUCCESSFULLY!
Journey: ('e1906618-b1e6-4649-a159-1e382cc483c6', 1) UPDATED SUCCESSFULLY!
Journey: ('e1906618-b1e6-4649-a159-1e382cc483c6', 1) RESUMED SUCCESSFULLY!


In [34]:
test.create_new_journey_version("e1906618-b1e6-4649-a159-1e382cc483c6", 1, 73)

AttributeError: 'JourneyBuilder' object has no attribute 'headers'

In [13]:
# version still persists although deleted (version 2 deleted but next version created is 3)
# json.dumps(data) to make it a string - that was the key

In [17]:
test.get_activities_by_id(id)

{'id': 'bcdf20a4-2050-4bb5-aae8-af108e87e766',
 'key': '5accfbcc-4a15-807f-3505-8ec4f28d2419',
 'name': 'test_kwr_multilanguage_setup',
 'lastPublishedDate': '2021-11-01T03:20:14',
 'description': '',
 'version': 1,
 'workflowApiVersion': 1.0,
 'createdDate': '2021-11-01T03:09:03.857',
 'modifiedDate': '2021-11-01T03:20:14.687',
 'activities': [{'id': 'c2f2209f-5ef8-49ab-8ab3-838fccda2b35',
   'key': 'MULTICRITERIADECISIONV2-1',
   'name': 'language',
   'description': '',
   'type': 'MULTICRITERIADECISION',
   'outcomes': [{'key': 'default_path_1',
     'next': 'RANDOMSPLITV2-1',
     'arguments': {},
     'metaData': {'label': 'lang = EN',
      'skipI18n': True,
      'isLabelFromConversion': False,
      'criteriaDescription': 'contact_surname contains A',
      'invalid': False}},
    {'key': '12033c12-af19-26d0-752f-2e18710dd5b4',
     'next': 'RANDOMSPLITV2-2',
     'arguments': {},
     'metaData': {'label': 'CA',
      'skipI18n': True,
      'isLabelFromConversion': False,
  

In [ ]:
"7a3ab8c6-e800-4faa-a867-0b59a4d37809"